In [1]:
# 1. Импорт библиотек
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
from collections import defaultdict
import pickle
import os
import pymorphy3

In [2]:
# 2. Загрузка данных
df = pd.read_csv('../shared_data/news.csv')

In [3]:
# 3. Инициализация pymorphy3 и стоп-слов
print("Инициализация pymorphy3...")
morph = pymorphy3.MorphAnalyzer()
nltk.download('stopwords')
russian_stopwords = set(stopwords.words('russian'))

Инициализация pymorphy3...


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# 4. Функция лемматизации с кэшированием для ускорения
class Lemmatizer:
    def __init__(self):
        self.morph = pymorphy3.MorphAnalyzer()
        self.cache = {}
    
    def lemmatize(self, word):
        if word not in self.cache:
            parsed = self.morph.parse(word)[0]
            self.cache[word] = parsed.normal_form
        return self.cache[word]

lemmatizer = Lemmatizer()

In [5]:
# 5. Функция предобработки с лемматизацией
def preprocess_text_with_lemmatization(text):
    """
    Предобработка текста с лемматизацией вместо стемминга
    """
    if pd.isna(text):
        return []
    
    # Очистка текста
    text = re.sub(r'[^a-zA-Zа-яА-ЯёЁ\s]', ' ', str(text))
    text = text.lower()
    
    # Токенизация
    tokens = text.split()
    
    # Удаление стоп-слов и лемматизация
    processed_tokens = []
    for token in tokens:
        if token not in russian_stopwords and len(token) > 2:
            lemma = lemmatizer.lemmatize(token)
            # Фильтрация слишком коротких лемм и служебных частей речи
            if len(lemma) > 2:
                processed_tokens.append(lemma)
    
    return processed_tokens

In [6]:
# 6. Построение инвертированного индекса с лемматизацией
def build_lemmatized_inverted_index(df, text_columns=['title', 'text']):
    """
    Построение инвертированного индекса с использованием лемматизации
    """
    inverted_index = defaultdict(dict)
    doc_lengths = {}
    
    print("Начинаем построение индекса с лемматизацией...")
    
    for idx, row in df.iterrows():
        if idx % 1000 == 0:
            print(f"Обработано {idx}/{len(df)} документов...")
        
        # Объединяем текст из указанных колонок
        combined_text = ' '.join(str(row[col]) for col in text_columns if col in row and pd.notna(row[col]))
        
        # Предобработка текста с лемматизацией
        tokens = preprocess_text_with_lemmatization(combined_text)
        doc_lengths[idx] = len(tokens)
        
        # Подсчет TF (Term Frequency) для документа
        term_freq = defaultdict(int)
        for token in tokens:
            term_freq[token] += 1
        
        # Добавление в инвертированный индекс
        for token, freq in term_freq.items():
            inverted_index[token][idx] = freq
    
    return inverted_index, doc_lengths

In [7]:
# 7. Функции для сохранения и загрузки лемматизированного индекса
def save_lemmatized_index(inverted_index, doc_lengths, filename='../shared_data/lemmatized_search_index.pkl'):
    """
    Сохранение лемматизированного индекса в файл
    """
    index_data = {
        'inverted_index': dict(inverted_index),
        'doc_lengths': doc_lengths,
        'type': 'lemmatized'
    }
    with open(filename, 'wb') as f:
        pickle.dump(index_data, f)
    print(f"Лемматизированный индекс сохранен в {filename}")

def load_lemmatized_index(filename='../shared_data/lemmatized_search_index.pkl'):
    """
    Загрузка лемматизированного индекса из файла
    """
    if os.path.exists(filename):
        with open(filename, 'rb') as f:
            index_data = pickle.load(f)
        print(f"Лемматизированный индекс загружен из {filename}")
        return defaultdict(dict, index_data['inverted_index']), index_data['doc_lengths']
    else:
        print(f"Файл {filename} не найден")
        return None, None

In [8]:
# 8. Построение и сохранение лемматизированного индекса
print("=== ПОСТРОЕНИЕ ИНДЕКСА С ЛЕММАТИЗАЦИЕЙ ===")

# Проверяем данные
print(f"Размер датасета: {len(df)} строк")
print(f"Колонки: {df.columns.tolist()}")

# Строим индекс с лемматизацией
lemmatized_index, lemmatized_doc_lengths = build_lemmatized_inverted_index(df)

# Сохраняем индекс
save_lemmatized_index(lemmatized_index, lemmatized_doc_lengths)

=== ПОСТРОЕНИЕ ИНДЕКСА С ЛЕММАТИЗАЦИЕЙ ===
Размер датасета: 21673 строк
Колонки: ['N', 'source', 'rubric', 'title', 'text']
Начинаем построение индекса с лемматизацией...
Обработано 0/21673 документов...
Обработано 1000/21673 документов...
Обработано 2000/21673 документов...
Обработано 3000/21673 документов...
Обработано 4000/21673 документов...
Обработано 5000/21673 документов...
Обработано 6000/21673 документов...
Обработано 7000/21673 документов...
Обработано 8000/21673 документов...
Обработано 9000/21673 документов...
Обработано 10000/21673 документов...
Обработано 11000/21673 документов...
Обработано 12000/21673 документов...
Обработано 13000/21673 документов...
Обработано 14000/21673 документов...
Обработано 15000/21673 документов...
Обработано 16000/21673 документов...
Обработано 17000/21673 документов...
Обработано 18000/21673 документов...
Обработано 19000/21673 документов...
Обработано 20000/21673 документов...
Обработано 21000/21673 документов...
Лемматизированный индекс сох

In [9]:
# 9. Статистика лемматизированного индекса
print(f"\n=== СТАТИСТИКА ЛЕММАТИЗИРОВАННОГО ИНДЕКСА ===")
print(f"Количество документов: {len(df)}")
print(f"Количество уникальных лемм: {len(lemmatized_index)}")


=== СТАТИСТИКА ЛЕММАТИЗИРОВАННОГО ИНДЕКСА ===
Количество документов: 21673
Количество уникальных лемм: 85308


In [10]:
# Топ-10 самых частых лемм
lemma_doc_freq = {lemma: len(docs) for lemma, docs in lemmatized_index.items()}
top_lemmas = sorted(lemma_doc_freq.items(), key=lambda x: x[1], reverse=True)[:10]
print(f"\nТоп-10 самых частых лемм:")
for lemma, freq in top_lemmas:
    print(f"  {lemma}: встречается в {freq} документах")


Топ-10 самых частых лемм:
  новость: встречается в 16021 документах
  риа: встречается в 15194 документах
  который: встречается в 13537 документах
  москва: встречается в 11071 документах
  год: встречается в 10844 документах
  россия: встречается в 9858 документах
  человек: встречается в 9231 документах
  также: встречается в 8984 документах
  это: встречается в 8722 документах
  страна: встречается в 7159 документах
